In [1]:
import duckdb

In [2]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [3]:
df = con.execute("""
                 SELECT *
                 FROM(
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-04-17'
                 ) WHERE row = 1
                """).fetchdf()

In [4]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final = df_final.rename(columns={"NATBR":"id"})
df_final = df_final.rename(columns={"MAKTX":"nm_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST":"vl_preco"})
df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500
1,10005,MACHADO,BT50,100,100
2,10003,PREGO,BT10,100,60
3,10001,PARAFUSO,BT10,100,100
4,10004,SERRA,BT50,100,200


In [5]:
df2 = df_final
df2 = df2.astype(
    {
        "id": "int32",
        "nm_produto": "string",
        "id_categoria": "string",
        "id_fornecedor": "int32",
        "vl_preco": "float"
    }
)
df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10005,MACHADO,BT50,100,100.0
2,10003,PREGO,BT10,100,60.0
3,10001,PARAFUSO,BT10,100,100.0
4,10004,SERRA,BT50,100,200.0


In [6]:
con.execute("""
            CREATE TABLE IF NOT EXISTS produtos (
                        id BIGINT,
                        nm_produto TEXT,
                        id_categoria TEXT,
                        id_fornecedor BIGINT,
                        vl_preco FLOAT
                        )
""")

In [7]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [8]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10005,MACHADO,BT50,100,100.0
2,10003,PREGO,BT10,100,60.0
3,10001,PARAFUSO,BT10,100,100.0
4,10004,SERRA,BT50,100,200.0


In [9]:
con.close()